In [1]:
import sys
#!pip install https://github.com/l0d0v1c/formulate/blob/main/dist/formulate-1.1-py3-none-any.whl
sys.path.append('../mix')

In [2]:
from components import components

In [3]:
c=components(physical={"∆Hf":True,"rho":None,"minor":None})

In [4]:
c.add("Water","H2O",{'∆Hf':-285.83,"rho":1.0,'minor':True})
c.add("Nitrogen","N2",{'∆Hf':0,"rho":0.01,'minor':False})
c.add("Oxygen","O2",{'∆Hf':0,"rho":0.01,'minor':False})
c.setrates({"Water":0.01,"Oxygen":0.19,'Nitrogen':0.8})
c.mixture

{'Water': {'H': 111.017,
  'O': 55.508,
  '∆Hf': -15865.97,
  'rho': 1.0,
  'minor': 1,
  'N': 0},
 'Nitrogen': {'N': 71.394,
  '∆Hf': 0.0,
  'rho': 0.01,
  'minor': 0,
  'H': 0,
  'O': 0},
 'Oxygen': {'O': 62.502, '∆Hf': 0.0, 'rho': 0.01, 'minor': 0, 'H': 0, 'N': 0}}

In [5]:
HT=components(physical={'∆Hf':True,'minor':None})
HT.add("TNT","C7H5N3O6",{"∆Hf":-67,'minor':False})
HT.add("RDX","C3H6N6O6",{"∆Hf":79,'minor':False})
HT.setrates({'TNT':0.75,'RDX':0.25})
HT2=components(physical={'∆Hf':True,'minor':None})
HT2.add("TNT","C7H5N3O6",{"∆Hf":-67,'minor':False})
HT2.add("RDX","C3H6N6O6",{"∆Hf":79,'minor':False})
HT2.setrates({'TNT':0.5,'RDX':0.5})
HT.mixing()

,Component,Rate,H,N,O,C,∆Hf,minor
0,TNT,0.75,22.01400,13.20800,26.41600,30.81900,-294.983,0
1,RDX,0.25,27.01300,27.01300,27.01300,13.50600,355.669,0
2,Formulation,1.00,23.26375,16.65925,26.56525,26.49075,-132.320,Non additive


In [6]:
from FSL import formulationsymboliclanguage    
                    
        

In [7]:
f=formulationsymboliclanguage([c,HT,HT2])
f.dict

{'Water': 'α', 'Nitrogen': 'A', 'Oxygen': 'B', 'TNT': 'C', 'RDX': 'D'}

In [8]:
f.max

{'Water': 0.01, 'Nitrogen': 0.8, 'Oxygen': 0.19, 'TNT': 0.75, 'RDX': 0.5}

In [9]:
f.min

{'Water': 0.01, 'Nitrogen': 0.8, 'Oxygen': 0.19, 'TNT': 0.5, 'RDX': 0.25}

In [10]:
f.encode([c,HT,HT2])

['ABα', 'CCCCCCCCCCCD', 'CDDDDDDDDDDD']

In [11]:
f.compos

{'Water': {'H': 111.017, 'O': 55.508, 'N': 0, 'minor': True},
 'Nitrogen': {'N': 71.394, 'H': 0, 'O': 0, 'minor': True},
 'Oxygen': {'O': 62.502, 'H': 0, 'N': 0, 'minor': True},
 'TNT': {'C': 30.819, 'H': 22.014, 'N': 13.208, 'O': 26.416, 'minor': True},
 'RDX': {'C': 13.506, 'H': 27.013, 'N': 27.013, 'O': 27.013, 'minor': True}}

In [12]:
f.decode(['CCCDα'])[0].formulationlist

,Component,Rate,minor
0,TNT,0.676,False
1,RDX,0.323,False
2,Water,0.001,True
3,Formulation,1.000,Non additive


In [13]:
f.decode(['ABα'])[0].formulationlist

,Component,Rate,minor
0,Nitrogen,0.807,False
1,Oxygen,0.192,False
2,Water,0.001,True
3,Formulation,1.000,Non additive


In [14]:
# cocktails

In [15]:
import pandas as pd
df=pd.read_excel("cocktails.xlsx")
df.head()

,Unnamed: 0,nom,categ,i1,d1,i2,d2,i3,d3,i4,d4,i5,d5,i6,d6
0,0,Gauguin,Cocktail Classics,Light Rum,2.0,Passion Fruit Syrup,1.0,Lemon Juice,1.00,Lime Juice,1.00,NaN,NaN,NaN,NaN
1,1,Fort Lauderdale,Cocktail Classics,Light Rum,1.5,Sweet Vermouth,0.5,Juice of Orange,0.25,Juice of a Lime,0.25,NaN,NaN,NaN,NaN
2,2,Apple Pie,Cordials and Liqueurs,Apple schnapps,3.0,Cinnamon schnapps,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Cuban Cocktail No. 1,Cocktail Classics,Juice of a Lime,0.5,Powdered Sugar,0.5,Light Rum,2.00,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Cool Carlos,Cocktail Classics,Dark rum,1.5,Cranberry Juice,2.0,Pineapple Juice,2.00,Orange curacao,1.00,Sour Mix,1.0,NaN,NaN


In [16]:
from collections import Counter
ingredients=[]
for i in range(1,7):
    for j in df[f"i{i}"].tolist():
        ingredients.append(j)
ingredients=Counter(ingredients)
composant={}
for name,cnt in ingredients.items():
    if cnt>30:
        composant[name]={'minor':False}
print(f"based on {len(composant)} ingredients")
listcompo=[]
for i,j in df.iterrows():
    try:
        cp=components(physical={"minor":None})
        rates={}
        for k in range(1,7):
            if j[f"d{k}"]==j[f"d{k}"] and j[f"i{k}"]==j[f"i{k}"] : #not nan
                name=j[f"i{k}"]
                if name in composant:
                    rate=j[f"d{k}"]
                    cp.add(name,"",{'minor':False})
                    rates[name]=rate
                else:
                    cp.add(name,"",{'minor':True})
                    rates[name]=0.001
                    
        cp.setrates(rates)
        cp.mixing()
    except:
        pass
    listcompo.append(cp)

based on 23 ingredients


In [17]:
listcompo[1].formulationlist

,Component,Rate,minor
0,Light Rum,0.666,0
1,Sweet Vermouth,0.222,0
2,Juice of Orange,0.000,1
3,Juice of a Lime,0.111,0
4,Formulation,1.000,Non additive


In [18]:
cocktails=formulationsymboliclanguage(listcompo)
cocktails.dict

Component limit reached for  Orange Juice, remaining major (13) and minor (0)
Component limit reached for  Apricot Flavored Brandy, remaining major (12) and minor (0)
Component limit reached for  White creme de menthe, remaining major (12) and minor (0)
Component limit reached for  Apricot Flavored Brandy, remaining major (12) and minor (0)
Component limit reached for  Apricot Flavored Brandy, remaining major (12) and minor (0)
Component limit reached for  Sloe gin, remaining major (12) and minor (0)
Component limit reached for  Green creme de menthe, remaining major (12) and minor (0)
Component limit reached for  Fresh mint leaves, remaining major (11) and minor (0)
Component limit reached for  Orange Juice, remaining major (11) and minor (0)
Component limit reached for  Tanqueray gin, remaining major (11) and minor (0)
Component limit reached for  Margarita Mix, remaining major (11) and minor (0)
Component limit reached for  Orange half-wheel, remaining major (11) and minor (0)
Compo

{' Light Rum': 'A',
 ' Passion Fruit Syrup': 'α',
 ' Lemon Juice': 'B',
 ' Lime Juice': 'β',
 ' Sweet Vermouth': 'C',
 ' Juice of Orange': 'γ',
 ' Juice of a Lime': 'D',
 ' Apple schnapps': 'δ',
 ' Cinnamon schnapps': 'ε',
 ' Powdered Sugar': 'E',
 ' Dark rum': 'ζ',
 ' Cranberry Juice': 'η',
 ' Pineapple Juice': 'F',
 ' Orange curacao': 'θ',
 ' Sour Mix': 'ι',
 ' Bourbon whiskey': 'G',
 ' Fresh lemon juice': 'H',
 ' Simple Syrup': 'I',
 ' Soda water': 'κ',
 ' Orange and lemon wheels, maraschino cherry': 'λ',
 ' cherry-flavored brandy': 'μ',
 ' Light cream': 'ν',
 ' Triple Sec': 'J',
 ' Maraschino': 'ξ',
 ' Creme de banana': 'ο',
 'Chilled Champagne': 'π',
 ' Amaretto': 'ρ',
 ' Scotch Whiskey': 'σ',
 ' Fresh carrot juice': 'τ',
 ' Tawny port': 'υ',
 ' Agave nectar': 'φ',
 ' Bacardi Rum': 'χ',
 ' Grenadine': 'K',
 ' Apple Brandy': 'ψ',
 ' Brandy': 'L',
 ' Gin': 'M',
 ' Anisette': 'ω',
 ' Dry Vermouth': 'N',
 ' Fresh Lime Juice': 'O',
 ' Orange Bitters': 'P',
 ' Bitters': 'Q',
 ' Egg Whit

In [19]:
encoded=cocktails.encode(listcompo)


component  Orange Juice not encodable
component  Apricot Flavored Brandy not encodable
component  White creme de menthe not encodable
component  Apricot Flavored Brandy not encodable
component  Apricot Flavored Brandy not encodable
component  Sloe gin not encodable
component  Green creme de menthe not encodable
component  Fresh mint leaves not encodable
component  Orange Juice not encodable
component  Tanqueray gin not encodable
component  Margarita Mix not encodable
component  Orange half-wheel not encodable
component  Green Chartreuse not encodable
component  Orange Juice not encodable
component  Green Chartreuse not encodable
component  Straight rye whiskey not encodable
component  Benedictine not encodable
component  Honey Syrup not encodable
component  Orange Juice not encodable
component  Green Chartreuse not encodable
component  Irish Whiskey not encodable
component  Ginger ale or soda water not encodable
component  Lemon twist (optional) not encodable
component  orgeat or almon

component  White creme de cacao not encodable
component  Creme de Cacao not encodable
component  Anis not encodable
component  Coffee-flavored brandy not encodable
component  half-and-half not encodable
component  151-Proof Rum not encodable
component  Blended whiskey not encodable
component  Fresh cherry tomatoes not encodable
component  Swedish Punch not encodable
component  Dubonnet not encodable
component  Maraschino liqueur not encodable
component  Applejack not encodable
component  Scotch not encodable
component  White creme de cacao not encodable
component  Galliano not encodable
component  Dubonnet not encodable
component  Maple Syrup not encodable
component  Yellow Chartreuse not encodable
component  Orange Juice not encodable
component  Sweet Sherry not encodable
component  Orange not encodable
component thin Peeled fresh ginger not encodable
component  Demerara Syrup not encodable
component  Fresh apple cider not encodable
component  Dry Sherry not encodable
component  17-ye

component  Grapefruit Juice not encodable
component  Orange blossom water not encodable
component  Peychaud's Bitters not encodable
component  Old Thompson Blended Whiskey not encodable
component  Old Mr. Boston Dry Gin not encodable
component  Benedictine not encodable
component  Hard cider not encodable
component Lime wedges not encodable
component  Whole strawberry not encodable
component  Rose water not encodable
component  Mezcal not encodable
component  Palo cortado or oloroso sherry not encodable
component  Allspice liqueur (pimento dram) not encodable
component  White creme de cacao not encodable
component  orange juice not encodable
component  Reposado tequila not encodable
component  White Curacao not encodable
component  Olive brine not encodable
component  Balsamic vinegar not encodable
component  Creme de Cassis not encodable
component  Campari not encodable
component  Grapefruit soda not encodable
component  Whole strawberry, hulled not encodable
component  Campari not en

In [20]:
print(encoded[1])
listcompo[1].formulationlist

AAAAAAAACCDγ


,Component,Rate,minor
0,Light Rum,0.666,0
1,Sweet Vermouth,0.222,0
2,Juice of Orange,0.000,1
3,Juice of a Lime,0.111,0
4,Formulation,1.000,Non additive


In [21]:
cocktails.decode([encoded[1]])[0].formulationlist

,Component,Rate,minor
0,Light Rum,0.624,False
1,Sweet Vermouth,0.228,False
2,Juice of a Lime,0.147,False
3,Juice of Orange,0.001,True
4,Formulation,1.000,Non additive
